In [1]:
! pip install transformers torch datasets evaluate wandb psutil

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
from datasets import load_dataset, load_metric
dataset = load_dataset("Sp1786/multiclass-sentiment-analysis-dataset")
# Function to clean text
import re
def clean_text(text):
    if text is None:
        return ""
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = text.lower()  # Convert to lowercase
    return text.strip()

# Function to clean the entire dataset
def clean_dataset(dataset):
    dataset = dataset.map(lambda x: {'text': clean_text(x['text'])})
    return dataset

cleaned_dataset = clean_dataset(dataset)
print(cleaned_dataset)

X_train = [example['text'] for example in cleaned_dataset['train']]
y_train = [example['label'] for example in cleaned_dataset['train']]
X_val = [example['text'] for example in cleaned_dataset['validation']]
y_val = [example['label'] for example in cleaned_dataset['validation']]
X_test = [example['text'] for example in cleaned_dataset['test']]
y_test = [example['label'] for example in cleaned_dataset['test']]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/31232 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5205 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5206 [00:00<?, ? examples/s]

Map:   0%|          | 0/31232 [00:00<?, ? examples/s]

Map:   0%|          | 0/5205 [00:00<?, ? examples/s]

Map:   0%|          | 0/5206 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'sentiment'],
        num_rows: 31232
    })
    validation: Dataset({
        features: ['id', 'text', 'label', 'sentiment'],
        num_rows: 5205
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'sentiment'],
        num_rows: 5206
    })
})


In [4]:
import torch
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, Trainer, TrainingArguments, EvalPrediction
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import datasets
import pandas as pd
from datasets import load_dataset, load_metric
import time
import psutil

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
# Tokenize function
def tokenize_func(data):
    return tokenizer(
            data['texts'],
            max_length=512,
            padding='max_length',
            return_attention_mask=True,
            truncation=True
        )
# Tokenize the Training Data
train_dataset = datasets.Dataset.from_pandas(pd.DataFrame({"texts":X_train,"labels":y_train}))
train_dataset = train_dataset.map(
    tokenize_func,
    batched=True,
    remove_columns=["texts"]
)
train_dataset

# Tokenize the Validation Data
val_dataset = datasets.Dataset.from_pandas(pd.DataFrame({"texts":X_val,"labels":y_val}))
val_dataset = val_dataset.map(
    tokenize_func,
    batched=True,
    remove_columns=["texts"]
)

val_dataset

# Tokenize the Test Data
test_dataset = datasets.Dataset.from_pandas(pd.DataFrame({"texts":X_test,"labels":y_test}))
test_dataset = test_dataset.map(
    tokenize_func,
    batched=True,
    remove_columns=["texts"]
)

test_dataset

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/31232 [00:00<?, ? examples/s]

Map:   0%|          | 0/5205 [00:00<?, ? examples/s]

Map:   0%|          | 0/5206 [00:00<?, ? examples/s]

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 5206
})

In [11]:
import torch
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, Trainer, TrainingArguments, EvalPrediction
from datasets import load_metric
import time
import wandb
import psutil

# Load tokenizer and model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)

# Freeze the original DistilBERT model parameters
for param in model.distilbert.parameters():
    param.requires_grad = False

# Create a prompt embedding layer
prompt_length = 10
prompt_embeddings = torch.nn.Embedding(prompt_length, model.config.hidden_size)
prompt_input_ids = torch.arange(prompt_length).expand((1, -1))

# Define the function for forward pass with prompt injection
class PTuningModel(torch.nn.Module):
    def __init__(self, model, prompt_embeddings):
        super().__init__()
        self.model = model
        self.prompt_embeddings = prompt_embeddings

    def forward(self, input_ids, attention_mask, labels=None):
        batch_size = input_ids.size(0)

        # Truncate input_ids to make room for the prompt embeddings
        max_input_length = 512 - prompt_length
        input_ids = input_ids[:, :max_input_length]
        attention_mask = attention_mask[:, :max_input_length]

        # Generate prompt embeddings for each batch
        prompt_embeds = self.prompt_embeddings(prompt_input_ids.to(input_ids.device))
        prompt_embeds = prompt_embeds.expand(batch_size, -1, -1)

        # Get input embeddings
        inputs_embeds = self.model.distilbert.embeddings.word_embeddings(input_ids)

        # Concatenate the prompt embeddings with the input embeddings
        inputs_embeds = torch.cat([prompt_embeds, inputs_embeds], dim=1)

        # Adjust the attention mask
        prompt_attention_mask = torch.ones((batch_size, prompt_length)).to(input_ids.device)
        attention_mask = torch.cat([prompt_attention_mask, attention_mask], dim=1)

        # Forward pass through the model
        outputs = self.model(
            inputs_embeds=inputs_embeds,
            attention_mask=attention_mask,
            labels=labels
        )

        return outputs

# Wrap the model with P-tuning
p_tuning_model = PTuningModel(model, prompt_embeddings)


# Define a function that can print the trainable parameters
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

# Print the percentage of trainable parameters
print(print_number_of_trainable_model_parameters(p_tuning_model))


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable model parameters: 600579
all model parameters: 66963459
percentage of trainable model parameters: 0.90%


In [ ]:
# Initialize a new wandb run
wandb.init(project="P-Tuning_Evaluation", entity="siyinggu-nyu")
# Set up wandb config
wandb.config = {
    "learning_rate": 5e-5,
    "epochs": 5,
    "batch_size": 16,
    "model_name": "distilbert-base-uncased",
    "dataset": "Sp1786/multiclass-sentiment-analysis-dataset"
}

# Define metrics
metric = load_metric("accuracy")

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return metric.compute(predictions=preds, references=p.label_ids)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    run_name='p-tuning-distilbert',
    evaluation_strategy="steps",
    eval_steps=10,
    save_strategy="steps",
    save_steps=10,
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    seed=42,
    report_to="wandb"
)


trainer = Trainer(
    model=p_tuning_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
start_time = time.time()
trainer.train()
end_time = time.time()

# Log total training time to wandb
wandb.log({"total_training_time": end_time - start_time})

# Print total training time
print({"total_training_time": end_time - start_time})

# Log memory usage
memory_info = psutil.virtual_memory()
wandb.log({"final_memory_usage": memory_info.used / (1024 ** 2)})  # Convert to MB
print({"final_memory_usage": memory_info.used / (1024 ** 2)})

# Evaluate the model
results = trainer.evaluate()
print(results)

# Finish the wandb run
wandb.finish()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss


In [ ]:
!pip install huggingface.huk > /dev/null 2>&1
from huggingface_hub import notebook_login
notebook_login()
model.push_to_hub("P-Tuning")
